# Prediction - Store Sales

# 1.0 Overview

### 1.1 Overview

Rossmann operates over 3,000 drug stores in 7 European countries. Currently, Rossmann store managers are tasked with predicting their daily sales for up to six weeks in advance. Store sales are influenced by many factors, including promotions, competition, school and state holidays, seasonality, and locality. With thousands of individual managers predicting sales based on their unique circumstances, the accuracy of results can be quite varied.

reference: https://www.kaggle.com/competitions/rossmann-store-sales/data

### 1.1.2 CRISP - DS

CRoss-Industry Process - Data Science

- Development Cicle Method 
   - Do the same process many times to improve the prediction in each cicle.
   - In each end-to-end cicle have a version with the solution.
   - Velocity to delivery values.
   - Mapping all possibibles problems.
   

#### 1.1.2.1 CRISP Steps

1. Business questions - Try to solve problems with all areas and people.
2. Understand the Business - What's the poeple want? Normally the people give the solution, and we need to discover the "pain" of this person/business
3. Data Collect - From Databases, API, Files...
4. Data Clean - Keep a "stadard" data and remove dirty data...
5. Data Exploration - What's the variable will impact the model(the result). Find the correlation between the fields.
6. Data Modeling - Prepare the data to put into the ML code (train and test, encoding, transform data, answer data...)
7. Apply ML 
8. Algoritimun Accuracy - Calculate the errors using ML's/Statistics (MAP, RME, MAPE, RMSE, ROCK, ACCURACY, TRUE TABLE...)

After the step 8º you and the team can see that the accuracy of the FIRST model will impact well the solution (normally not happen after the first cicle), we can put the MODEL in production, if not, start the other CICLE in STEP 2.

**IMPORTANT** you not back on the step 1 because the problem will be YOU, and we need to decide and change the steps 2º to 8º.

#### 1.1.2.2 Business Questions

What's the sales value of each store in the next 6 weeks?

#### 1.1.2.3 Understand the Business

- **What's the motivation to do this?** CFO (Chief Financial Officer) has requested the prediction of sales at the monthly meeting about the stores results.
- **What's the main cause of the problem?** Difficult to understand the value of investment to renovate each store.
- **Who is the stakeholder of the problem?** Director Finance (CFO) of Rossmann
- **What's the solution/product format that will be delivered?**
    - Granularity: Prediction sell per day and store on the next 42 days (6 weeks).
    - Problem type: Forecast sell .
    - Potencials Methods: Temporal series, regression with modifications.
    - Product/solution format: Check on the phone the total salles in each store on the final of the 6 weeks

# 2.0 IMPORTS

In [29]:
import pandas as pd
import inflection

# 3.0 Data clean

- **Why Data Description?**
    - **How challenger is the problem im working?**
    - **What's the duration of this problem?**
        - Years?
        - Months?
        - Days?
    - **What's the quantity of the data?**
        - I have the enought resourcing to work?
            - I need servers to process the data?
            - Clusters?
            - Spark?
            - Hadoop?
    - **What's the variable types (will help to decide the techniques to apply):**
        - % Numeric?
        - % Categories?
        - % Temporal?
    - **What's the NA(non-applicable/empty) data?**
        - NA's volume
        - What's the reazon of this NA's? 
            - I can do the job?
            - How I will reaplace the data?
            - I can replace or delete the data?
            - **details** sometimes the system isn't recording the data, the input is manually.
    - **Data summary:**
        - Describe stats.
        - Get the limits, minimum, maximun, STD, mean...
        
    

### 3.0.1 Helper Functions

### 3.0.2 Loading Data

In [2]:
df_sales_raw = pd.read_csv( '../data/train.csv', low_memory=False )
df_store_raw = pd.read_csv( '../data/store.csv', low_memory=False )

# Merge the both raw data
# merge(reference_table, merged_table, type of merge, column key equal in both tables )
df_raw = pd.merge(df_sales_raw, df_store_raw, how="left", on="Store")

In [3]:
#Sample of sales data
df_sales_raw.sample()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
957765,766,6,2013-02-23,5503,509,1,0,0,0


In [4]:
#Sample of store data
df_store_raw.sample()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
246,247,d,c,70.0,11.0,2010.0,1,5.0,2013.0,"Feb,May,Aug,Nov"


In [5]:
#Sample of sales data
df_raw.sample()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
743214,295,2,2013-09-03,4537,669,1,0,0,1,a,a,210.0,11.0,2000.0,1,36.0,2013.0,"Mar,Jun,Sept,Dec"


## 3.1 DATA DESCRIPTION

#### 3.1.1 Rename Columns

In [18]:
#Copy to keep the raw data.
df1 = df_raw.copy()

In [19]:
#Show the columns name
df1.columns

Index(['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment',
       'CompetitionDistance', 'CompetitionOpenSinceMonth',
       'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
       'Promo2SinceYear', 'PromoInterval'],
      dtype='object')

In [20]:
cols_old_names = df1.columns

#Create a function using the inflection library with undersocore method to puth _ between the name of the columns
snakecase = lambda x: inflection.underscore(x)

#Create a list with new columns name
cols_new_name = list(map(snakecase, cols_old_names))

#replace the old to new columns names
df1.columns = cols_new_name

#Show the new names
df1.columns

Index(['store', 'day_of_week', 'date', 'sales', 'customers', 'open', 'promo',
       'state_holiday', 'school_holiday', 'store_type', 'assortment',
       'competition_distance', 'competition_open_since_month',
       'competition_open_since_year', 'promo2', 'promo2_since_week',
       'promo2_since_year', 'promo_interval'],
      dtype='object')

#### 3.1.2 Data Dimensions

In [22]:
#Total rows and columns
print('Number of Rows: {}'.format(df1.shape[0]))
print('Number of Columns: {}'.format(df1.shape[1]))

Number of Rows: 1017209
Number of Columns: 18


#### 3.1.2 Data Types

In [26]:
#Show the data type for each column
df1.dtypes

store                                    int64
day_of_week                              int64
date                            datetime64[ns]
sales                                    int64
customers                                int64
open                                     int64
promo                                    int64
state_holiday                           object
school_holiday                           int64
store_type                              object
assortment                              object
competition_distance                   float64
competition_open_since_month           float64
competition_open_since_year            float64
promo2                                   int64
promo2_since_week                      float64
promo2_since_year                      float64
promo_interval                          object
dtype: object

In [25]:
#Change the date object to datetime type
df1['date'] = pd.to_datetime( df1['date'] )

#### 2.3.2.4 Check NA's

In [28]:
#Check the NA's total has in each line
df1.isna().sum()

store                                0
day_of_week                          0
date                                 0
sales                                0
customers                            0
open                                 0
promo                                0
state_holiday                        0
school_holiday                       0
store_type                           0
assortment                           0
competition_distance              2642
competition_open_since_month    323348
competition_open_since_year     323348
promo2                               0
promo2_since_week               508031
promo2_since_year               508031
promo_interval                  508031
dtype: int64

**important** We have three possibilities to management the NA's
  1. Discard the row that contain NA's (delete)
  2. Use the mean, median, or ML methods.
  3. Use the business questions to decide how we can replace.